In [1]:
# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [2]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [3]:
# os.environ['WANDB_NOTEBOOK_NAME'] = 'RUN_1'
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lihem (takim). Use `wandb login --relogin` to force relogin


True

In [4]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'augmentation': {
          'values': ['none', 'light', 'heavy']
        }
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100},
    'learning_rate': {
        'value': 0.00025118864
        },
    'batch_size': {
          'value': 64
        },
    'kernel_size': {
        'value': (3, 3)
        },
    'dropout': {
          'value': True
        },
    'pooling': {
          'value': 'max'
        },
    'batchnorm': {
          'value': True
        },
    'a_layers': {
          'value': 16
        },
    'reg_alpha': {
        'value': 0
        },
    'normalization': {
        'value': False}
    })

In [5]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'a_layers': {'value': 16},
                'augmentation': {'values': ['none', 'light', 'heavy']},
                'batch_size': {'value': 64},
                'batchnorm': {'value': True},
                'dropout': {'value': True},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'kernel_size': {'value': (3, 3)},
                'learning_rate': {'value': 0.00025118864},
                'normalization': {'value': False},
                'pooling': {'value': 'max'},
                'reg_alpha': {'value': 0}}}


In [6]:
sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: s4w5tx8e
Sweep URL: https://wandb.ai/takim/CIFAR-10_Classification/sweeps/s4w5tx8e


In [7]:
import tensorflow as tf

def create_model(kernel_size, dropout, pooling, batchnorm, n_layers, reg_alpha):
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(64, kernel_size, activation='relu', padding='same', input_shape=(32, 32, 3), kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(64, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(128, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(128, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(256, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(256, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(256, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    if n_layers == 19:
        model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
        model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
        model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
        if batchnorm:
            model.add(tf.keras.layers.BatchNormalization())


    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(units=4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(units=4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    return model


In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        do_normalization = config['normalization']
        do_augmentation = config['augmentation'] != 'none'

        x_train_to_use = (x_train.astype('float32') / 255) if do_normalization else x_train
        x_test_to_use = (x_test.astype('float32') / 255) if do_normalization else x_test

        tf.keras.backend.clear_session()
        model = create_model(config["kernel_size"], config["dropout"], config["pooling"], config["batchnorm"], config["a_layers"], config["reg_alpha"])
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        if not do_augmentation:
            history = model.fit(x_train_to_use, y_train,
                                epochs=config["epochs"],
                                batch_size=config["batch_size"],
                                validation_split=0.1,
                                callbacks=[
                                    WandbMetricsLogger(log_freq='epoch'),
                                    early_stopping
                                ], verbose=1
                                )
        else:
            if config['augmentation'] == 'light':
                datagen = ImageDataGenerator(
                    rotation_range=20,
                    horizontal_flip=True,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    fill_mode='nearest'
                )
            else:
                datagen = ImageDataGenerator(
                    rotation_range=40,
                    horizontal_flip=True,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.1,
                    zoom_range=0.1,
                    fill_mode='nearest'
                )

            x_tr, x_vl, y_tr, y_vl = train_test_split(x_train_to_use, y_train, test_size=0.1, random_state=42)

            train_datagen = datagen.flow(x_tr, y_tr, batch_size=config["batch_size"])
            history = model.fit(train_datagen,
                                epochs=config["epochs"],
                                batch_size=config["batch_size"],
                                validation_data=(x_vl, y_vl),
                                callbacks=[
                                    WandbMetricsLogger(log_freq='epoch'),
                                    early_stopping
                                ], verbose=1
                                )
            
        
        test_stats = model.evaluate(x_test_to_use, y_test)
        wandb.log({"test_loss": test_stats[0]})
        wandb.log({"test_acc": test_stats[1]})

        val_loss_history = history.history['val_loss']
        val_acc_history = history.history['val_accuracy']

        best_epoch_num = -1 if (len(val_loss_history) == 100 or len(val_loss_history) <= 10) else (len(val_loss_history) - 11)

        wandb.log({"best_val_loss": val_loss_history[best_epoch_num]})
        wandb.log({"best_val_acc": val_acc_history[best_epoch_num]})

In [9]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: njsfs215 with config:
wandb: 	a_layers: 16
wandb: 	augmentation: none
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: [3, 3]
wandb: 	learning_rate: 0.00025118864
wandb: 	normalization: False
wandb: 	pooling: max
wandb: 	reg_alpha: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 27s 34ms/step - loss: 1.6127 - accuracy: 0.4149 - top@3_accuracy: 0.7568 - val_loss: 1.3176 - val_accuracy: 0.5268 - val_top@3_accuracy: 0.8308
Epoch 2/100
704/704 [==============================] - 23s 32ms/step - loss: 1.0666 - accuracy: 0.6299 - top@3_accuracy: 0.8860 - val_loss: 0.9793 - val_accuracy: 0.6734 - val_top@3_accuracy: 0.9024
Epoch 3/100
704/704 [==============================] - 23s 32ms/step - loss: 0.8147 - accuracy: 0.7252 - top@3_accuracy: 0.9259 - val_loss: 1.2230 - val_accuracy: 0.6204 - val_top@3_accuracy: 0.8370
Epoch 4/100
704/704 [==============================] - 23s 32ms/step - loss: 0.6498 - accuracy: 0.7860 - top@3_accuracy: 0.9475 - val_loss: 1.0653 - val_accuracy: 0.6718 - val_top@3_accuracy: 0.8922
Epoch 5/100
704/704 [==============================] - 23s 32ms/step - loss: 0.5421 - accuracy: 0.8234 - top@3_accuracy: 0.9606 - val_loss: 0.6598 - val_accuracy: 0.7804 - val_top@3_accuracy: 0.9458
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▄▅▆▆▇▇▇▇█▇▇███
epoch/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▄▃▃▂▂▂▁▂▂▁▁▁
epoch/top@3_accuracy,▁▅▆▇▇▇▇███▇████
epoch/val_accuracy,▁▅▃▄▇▇▆▇▇█▇███▇
epoch/val_loss,█▄▇▅▁▂▄▂▃▂▂▂▃▂▄
epoch/val_top@3_accuracy,▁▅▁▄▇▇▅▇▇█▇███▇
test_acc,▁


wandb: Agent Starting Run: 73x9wrfn with config:
wandb: 	a_layers: 16
wandb: 	augmentation: light
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: [3, 3]
wandb: 	learning_rate: 0.00025118864
wandb: 	normalization: False
wandb: 	pooling: max
wandb: 	reg_alpha: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 24s 33ms/step - loss: 1.7191 - accuracy: 0.3797 - top@3_accuracy: 0.7291 - val_loss: 1.2887 - val_accuracy: 0.5336 - val_top@3_accuracy: 0.8508
Epoch 2/100
704/704 [==============================] - 23s 32ms/step - loss: 1.2531 - accuracy: 0.5540 - top@3_accuracy: 0.8528 - val_loss: 1.2308 - val_accuracy: 0.5816 - val_top@3_accuracy: 0.8654
Epoch 3/100
704/704 [==============================] - 23s 32ms/step - loss: 1.0372 - accuracy: 0.6399 - top@3_accuracy: 0.8907 - val_loss: 1.2089 - val_accuracy: 0.6182 - val_top@3_accuracy: 0.8412
Epoch 4/100
704/704 [==============================] - 23s 32ms/step - loss: 0.9037 - accuracy: 0.6949 - top@3_accuracy: 0.9114 - val_loss: 0.8746 - val_accuracy: 0.6998 - val_top@3_accuracy: 0.9256
Epoch 5/100
704/704 [==============================] - 23s 32ms/step - loss: 0.8173 - accuracy: 0.7276 - top@3_accuracy: 0.9248 - val_loss: 0.8146 - val_accuracy: 0.7250 - val_top@3_accuracy: 0.9298
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇███▇████████
epoch/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▄▅▆▆▇▇▇▇▇▇▇▇▇▇█▇██▇████▇█████████████
epoch/val_accuracy,▁▂▃▄▅▆▆▆▆▇▇▇▆▆▇▇▇▆▇▇██▄▆▇▇▇██▇██████▇▃
epoch/val_loss,██▇▅▄▄▃▃▃▃▃▂▃▃▂▂▂▃▂▂▁▁█▃▂▂▂▁▁▂▁▁▁▁▁▁▂▇
epoch/val_top@3_accuracy,▁▂▁▅▅▆▆▆▇▇▇▇▆▇▇▇▇▇▇▇██▅▆▇▇▇██▇▇█████▇▂
test_acc,▁


wandb: Agent Starting Run: i4ksnhzy with config:
wandb: 	a_layers: 16
wandb: 	augmentation: heavy
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: [3, 3]
wandb: 	learning_rate: 0.00025118864
wandb: 	normalization: False
wandb: 	pooling: max
wandb: 	reg_alpha: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 24s 33ms/step - loss: 1.8729 - accuracy: 0.3126 - top@3_accuracy: 0.6684 - val_loss: 1.6007 - val_accuracy: 0.4124 - val_top@3_accuracy: 0.7570
Epoch 2/100
704/704 [==============================] - 23s 32ms/step - loss: 1.4870 - accuracy: 0.4588 - top@3_accuracy: 0.7964 - val_loss: 1.5486 - val_accuracy: 0.4586 - val_top@3_accuracy: 0.7800
Epoch 3/100
704/704 [==============================] - 23s 32ms/step - loss: 1.3153 - accuracy: 0.5336 - top@3_accuracy: 0.8374 - val_loss: 1.3373 - val_accuracy: 0.5216 - val_top@3_accuracy: 0.8252
Epoch 4/100
704/704 [==============================] - 23s 32ms/step - loss: 1.1802 - accuracy: 0.5894 - top@3_accuracy: 0.8654 - val_loss: 1.0877 - val_accuracy: 0.6194 - val_top@3_accuracy: 0.8780
Epoch 5/100
704/704 [==============================] - 23s 33ms/step - loss: 1.0740 - accuracy: 0.6322 - top@3_accuracy: 0.8834 - val_loss: 1.1803 - val_accuracy: 0.6206 - val_top@3_accuracy: 0.8884
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇▇▇██▇████▇█
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▃▂▂▂▁▁▂▂▂▂▁▁▂▁▁▁▁▂▁
epoch/top@3_accuracy,▁▄▅▅▆▆▇▇▇▇▇▇▇▇▇▇█▇▇██▇█▇████████████████
epoch/val_accuracy,▁▂▃▄▄▅▅▅▅▇▆▆▇▇▇▆▇▇▇▇█▇▇▆▇▇██▇▄██████████
epoch/val_loss,██▆▅▆▄▅▄▄▂▃▄▃▃▃▃▂▃▂▂▁▂▂▃▂▂▁▁▂▆▁▁▁▁▂▂▁▁▁▁
epoch/val_top@3_accuracy,▁▂▃▅▅▆▆▅▅▇▇▆▇▇▇▇▇▇▇██▇█▇█████▂██████████
test_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
